In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
# from constants import *


from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam

In [2]:
import tensorflow as tf
import keras

In [3]:
# tf.debugging.set_log_device_placement(True)

In [4]:
MAX_LEN_TEXT = 150
MAX_LEN_SUMMARY = 100

#### reading data

In [5]:
x_train = pd.read_csv(r"U:\nlp_project\text_summarization\artifacts\data\preprocessed_data\x_tr.csv")
y_train = pd.read_csv(r"U:\nlp_project\text_summarization\artifacts\data\preprocessed_data\y_tr.csv")

In [6]:
x_val = pd.read_csv(r"U:\nlp_project\text_summarization\artifacts\data\preprocessed_data\x_val.csv")
y_val = pd.read_csv(r"U:\nlp_project\text_summarization\artifacts\data\preprocessed_data\y_val.csv")

In [7]:
print(type(x_train["cleaned_text"]))
print(type(y_train["cleaned_summary"]))
print("\n")
print(type(x_val["cleaned_text"]))
print(type(y_val["cleaned_summary"]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


#### prepairing Tokenizer

- Text Tokenizer

In [8]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_train["cleaned_text"]))

#convert text sequences into integer sequences
x_train    =   x_tokenizer.texts_to_sequences(list(x_train["cleaned_text"])) 
x_val   =   x_tokenizer.texts_to_sequences(list(x_val["cleaned_text"]))

#padding zero upto maximum length
# x_train    =   pad_sequences(x_train,  maxlen=MAX_LEN_TEXT, padding='post') 
# x_val   =   pad_sequences(x_val, maxlen=MAX_LEN_TEXT, padding='post')

# x_voc_size   =  len(x_tokenizer.word_index) +1

In [10]:
max([len(l) for l in x_train])

858

In [11]:
max([len(l) for l in x_val])

472

In [ ]:
x_train.shape

In [ ]:
x_train.shape,x_val.shape,x_voc_size

- summary tokenizer

In [10]:
#preparing a tokenizer for summary on training data 
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_train["cleaned_summary"]))

#convert summary sequences into integer sequences
y_train    =   y_tokenizer.texts_to_sequences(list(y_train["cleaned_summary"])) 
y_val   =   y_tokenizer.texts_to_sequences(list(y_val["cleaned_summary"])) 

# #padding zero upto maximum length
# y_train    =   pad_sequences(y_train, maxlen=MAX_LEN_SUMMARY, padding='post')
# y_val   =   pad_sequences(y_val, maxlen=MAX_LEN_SUMMARY, padding='post')

# y_voc_size  =   len(y_tokenizer.word_index) +1

In [12]:
max([len(l) for l in y_train])

15

In [13]:
max([len(l) for l in y_val])

15

In [ ]:
y_train.shape,y_val.shape,y_voc_size

In [ ]:
y_train[0]

In [13]:
# import numpy as np
# print(np.isnan(x_train).any(), np.isinf(x_train).any())
# print(np.isnan(y_tr).any(), np.isinf(y_tr).any())

##### save the tokenizers:

In [14]:
import pickle

In [ ]:
# CODE TO SAVE TOKENIZER
y_tokenizer,x_tokenizer

In [16]:
from keras import backend as K
K.clear_session()

In [ ]:
import tensorflow as tf

# Enable memory growth for GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus[0].device_type:
    try:
        for gpu in gpus:
            tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=1024*2)])
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)  # This will catch the error if it's already initializeda
        

#### Building Model:

In [18]:
import tensorflow as tf

In [19]:
from keras import backend as K 
K.clear_session() 

#### 1st Without Attention:

In [20]:
# model=Sequential()
# model.add(Embedding(x_voc_size,500,input_length=100))
# model.compile('adam','mse')

In [21]:
# prediction = model.predict(x_train[:10])

In [22]:
# prediction.shape
#o/p: (100, 500)

In [23]:
# Embedding(vocab_size,output_dimension,input_len)
#o/p:(batch_size,each_seq_len,each_token_is_represented_by_dimension)

In [24]:
latent_dim = 500 
class create_encoder(keras.Model):
    def __init__(self,encoder_inputs,x_voc_size):
        super().__init__()
        self.encoder_emb = Embedding(x_voc_size, latent_dim,trainable=True,name="encoder_emb")(encoder_inputs)
        self.encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,name="lstm1")
        self.dropout_one = Dropout((0.2),name="dropout_one")
        self.encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,name="lstm2")
        self.dropout_two = Dropout((0.2),name="dropout_two")
        self.encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,name="lstm3")

    def call(self,training):
        encoder_output1, state_h1, state_c1 = self.encoder_lstm1(self.encoder_emb)
        x = self.dropout_one(encoder_output1, training=training)
        encoder_output2, state_h2, state_c2 = self.encoder_lstm2(x)
        x = self.dropout_two(encoder_output2, training=training)
        encoder_outputs, state_h, state_c = self.encoder_lstm3(x)
        return [state_h,state_c]
    

class create_decoder(keras.Model):
    def __init__(self,decoder_inputs):
        super().__init__()
        self.decoder_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True,name="decoder_emb")(decoder_inputs)
        self.decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,name="decoder_lstm") 
        self.dropout_decoder = Dropout((0.2),name="dropout_decoder")
        self.decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
        
    def call(self,context_vector,training):
        decoder_outputs,decoder_fwd_state, decoder_back_state = self.decoder_lstm(self.decoder_emb_layer,initial_state=[context_vector[0], context_vector[1]])
        x = self.dropout_decoder(decoder_outputs, training=training)
        decoder_output = self.decoder_dense(x)
        return decoder_output

In [25]:
# latent_dim = 500 
# class create_encoder(keras.Model):
#     def __init__(self,encoder_inputs,x_voc_size):
#         super().__init__()
#         self.encoder_emb = Embedding(x_voc_size, latent_dim,trainable=True,name="encoder_emb")(encoder_inputs)
#         self.encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,name="lstm1")
#         self.dropout = Dropout((0.2))
#         self.encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,name="lstm2")
#         self.encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,name="lstm3")

#     def call(self,training=False):
#         encoder_output1, state_h1, state_c1 = self.encoder_lstm1(self.encoder_emb)
#         x = self.dropout(encoder_output1, training=training)
#         encoder_output2, state_h2, state_c2 = self.encoder_lstm2(encoder_output1)
#         x = self.dropout(encoder_output1, training=training)
#         encoder_outputs, state_h, state_c = self.encoder_lstm3(encoder_output2)
#         return [state_h,state_c]
    

# class create_decoder(keras.Model):
#     def __init__(self,decoder_inputs):
#         super().__init__()
#         self.decoder_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True,name="decoder_emb")(decoder_inputs)
#         self.decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,name="decoder_lstm") 
#         self.decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
        
#     def call(self,context_vector):
#         decoder_outputs,decoder_fwd_state, decoder_back_state = self.decoder_lstm(self.decoder_emb_layer,initial_state=[context_vector[0], context_vector[1]])
#         decoder_output = self.decoder_dense(decoder_outputs)
#         return decoder_output

In [26]:
encoder_inputs = Input(MAX_LEN_TEXT)
# encoder_inputs = Input(shape=(MAX_LEN_TEXT,))
encoder = create_encoder(encoder_inputs,x_voc_size)
context_vector = encoder.call(True)

decoder_inputs = Input(shape=(None,)) 
decoder = create_decoder(decoder_inputs)
decoder_output = decoder.call(context_vector, True)

In [27]:
model = Model([encoder_inputs, decoder_inputs], decoder_output)

In [28]:
initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
model.summary()

In [ ]:
# This code require graphviz to run
from keras.utils import plot_model
plot_model(model=model, show_shapes=True, show_layer_names=True)

In [ ]:
model.layers

#### start model training

In [32]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(moniter="val_loss", factor=0.5,patience=2,minimum_lr='1e-6')
callbacks = [es,reduce_lr]

In [ ]:
y_train.shape

In [ ]:
import numpy as np
test = [[1,2,3,4,5,6],[11,12,13,14,15,16],[21,22,23,24,25,26]]
np.array(test)

In [ ]:
y_train.shape

In [ ]:
device_spec = tf.DeviceSpec(job ="localhost", replica = 0, device_type = "GPU")

# Enabling device logging
# tf.debugging.set_log_device_placement(True)
 
# Specifying the device
with tf.device(device_spec):
  # print('Device Spec: ', device_spec.to_string())
  history = model.fit([x_train, y_train[:, :-1]], 
                    y_train.reshape(y_train.shape[0], y_train.shape[1], 1)[:, 1:], 
                    epochs=100, 
                    callbacks=callbacks, 
                    batch_size=50,
                    validation_data=([x_val, y_val[:, :-1]], 
                                     y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:, 1:]), verbose=True)

In [ ]:
from matplotlib import pyplot 
pyplot.plot(history.history['loss'], label='train') 
pyplot.plot(history.history['val_loss'], label='val') 
pyplot.legend() 
pyplot.show()

# save tokenizers, and model:

In [38]:
import pickle
import os

In [39]:
# model.save(r"U:\nlp_project\text_summarization\artifacts\model\ts_model.pickle")

In [40]:
# x_tokenizer_dir = os.path.join(r'U:\nlp_project\text_summarization\artifacts\tokenizer', 'x')
# y_tokenizer_dir = os.path.join(r'U:\nlp_project\text_summarization\artifacts\tokenizer', 'y')
# if not os.path.exists(x_tokenizer_dir) and not os.path.exists(x_tokenizer_dir):
#     os.mkdir(x_tokenizer_dir)
#     os.mkdir(y_tokenizer_dir)

In [41]:
# x_tokenizer_path = os.path.join(x_tokenizer_dir, 'x_tokenizer') + '.pkl'
# with open(x_tokenizer_path, 'wb') as file:
#     pickle.dump(x_tokenizer, file)

# y_tokenizer_path = os.path.join(y_tokenizer_dir, 'y_tokenizer') + '.pkl'
# with open(y_tokenizer_path, 'wb') as file:
#     pickle.dump(y_tokenizer, file)